In [1]:
from langchain.agents.self_ask_with_search.prompt import PROMPT

from langchain.tools import StructuredTool   # As for email input

from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain.chains.graph_qa.cypher_utils import CypherQueryCorrector

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pydantic import BaseModel, Field
from langchain_core.tools import tool

from langchain.prompts import PromptTemplate

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_models import ChatOllama

from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.schema import SystemMessage, HumanMessage

from dotenv import load_dotenv
from ast import literal_eval
import json
import os

## This is the first agent

Enhanced accuracy of the GraphCypherQAChain: https://python.langchain.com/docs/tutorials/graph/

In [2]:
# Initialize LLM
api_key = os.getenv("GOOGLE_API_KEY")

# Set it as an environment variable (LangChain uses this)
os.environ["GOOGLE_API_KEY"] = api_key

llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [3]:
# Load environment variables
load_dotenv()

# # Initialize local LLM 
# llm = ChatOllama(model="deepseek-r1", streaming=True)
# llm_llama_3 = ChatOllama(model="llama3", streaming=True)

# Connect to Neo4j
neo4j_pass = os.getenv("NEO4J_PASSWORD")

graph = Neo4jGraph(
    url="neo4j://127.0.0.1:7687",
    username="neo4j",
    password=neo4j_pass,
    enhanced_schema=True
)

C:\Users\PC\AppData\Local\Temp\ipykernel_17324\4058364274.py:11: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS re

In [4]:
CYPHER_GENERATION_TEMPLATE = """
Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question}"""


# You are given context data from a graph database query. Extract the relevant information and return it as a JSON object. 
# Only include the fields that are present in the context. Do not include any explanation or extra text—return only valid JSON.

QA_GENERATION_TEMPLATE = """
       Task: answer the question you are given based on the context provided.
       Instructions:
       - COMPLETELY TRUST the email, project, Technology, Person name that are returned as the correct results for that person, EVEN IF the results does not contain their name
       - You are an assistant that helps to form nice and human understandable answers.
       - If ask for people information like email, role then return their name too
       - Use the context information provided to generate a well organized and comprehensve answer to the user's question. 
        Please extract and return the email address(es) that answer the question. .
        You must use the information to construct your answer. 
        The provided information is authoritative; do not doubt it or try to use your internal knowledge to correct it. 
        Make the answer sound like a response to the question without mentioning that you based the result on the given information. 
        If there is no information provided, say that the knowledge base returned empty results.

        Here's the information:
        {context}

        Question: {question}
        Answer:
            """

CYPHER_GENERATION_PROMPT = PromptTemplate(input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE)

prompt_qa = PromptTemplate(input_variables=["context", "question"], template=QA_GENERATION_TEMPLATE)

# Create GraphCypherQAChain
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm_gemini,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    qa_prompt=prompt_qa,
    return_intermediate_steps=True,    # See the detail information of though process
    # return_direct=True                # Return directly the result found (in JSON form)
)

# Wrap into a tool
graph_tool = Tool(
    name="GraphCypherQA",
    func=cypher_chain.invoke,
    description=(
        "Use this tool to answer questions about people, projects, and technologies"
    )
)


# User Query Testing
# response = cypher_chain.invoke("Find Alice email")
# print(response)


In [9]:
from typing import List, Dict

@tool
def get_graph_context(question: str) -> str:
    """
    Run a question through the graph QA chain and return the person's information in natural language.
    """
    response = cypher_chain.invoke(question)
    # context = response.get("intermediate_steps", [{}])[1].get("context", [])

    # if not context:
    #     return "No information found for the question."

    # # Flatten and deduplicate values
    # unique_items = {tuple(sorted(d.items())) for d in context}
    # unique_dicts = [dict(t) for t in unique_items]

    # info_lines = []
    # for entry in unique_dicts:
    #     entry_info = ", ".join(f"{k} is {v}" for k, v in entry.items())
    #     info_lines.append(entry_info)

    # return f"Here's the information I found about the person you asked about: their {', and '.join(info_lines)}."
    return response.get("result", "No result found.")

# Make sure to have a clear Prompt
# context = get_graph_context.invoke("Find Bob's friend, and then tell me if they are also connected to a project and what that project is.")
# print(context)

## Sending Email

In [5]:
# agent using tool working fine
system_message_email = """
You are an AI assistant that helps send emails.

Use the tool 'send_email' when the user asks to contact someone.
Make sure to call the tool using the format:
send_email(to="...", subject="...", body="...")

Do not use positional arguments. Only use named arguments.
"""

class EmailInput(BaseModel):
    to: str = Field(description="Recipient email")
    subject: str = Field(description="Title of the Email")
    body: str = Field(description="Email content")

@tool(args_schema=EmailInput)
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to a recipient with subject and body."""
    sender_email = os.getenv("EMAIL_USER")
    password = os.getenv("EMAIL_PASSWORD")
    
    try:
        msg = MIMEMultipart()
        msg["From"] = sender_email
        msg["To"] = to
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain"))

        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, password)
            server.sendmail(sender_email, to, msg.as_string())
        
        return f"Email sent to {to} with subject '{subject}'"
    except Exception as e:
        return f"Failed to send email: {e}"


email_agent = initialize_agent(
    tools=[send_email],
    llm=llm_gemini,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    agent_kwargs={"system_message": system_message_email},
    verbose=True
)

# Agent as a Tool
email_agent_tool = Tool(
    name="email_agent_tool",
    func=email_agent.invoke,
    description=(
        "Send an email. Use when the user asks to contact or notify someone."
        )
)


C:\Users\PC\AppData\Local\Temp\ipykernel_17324\1179464683.py:40: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  email_agent = initialize_agent(


In [ ]:
print(email_agent_tool.args)
print(send_email.args)

In [ ]:
# agent using tool with 1 input
# system_message_email = """
# You are an AI assistant that helps send emails.

# Use the tool 'send_email' when the user asks to contact someone.
# Make sure to call the tool using the format:
# send_email('{"to": "recipient@example.com", "subject": "Your Subject", "body": "Your message content."}')

# Do not use positional or named arguments separately.
# Always provide a single JSON string as the argument to the tool.
# """
system_message_email = """
You are an AI assistant that helps send emails.

Use the tool 'send_email' when the user asks to contact someone.
Important:
- `send_email` only accepts **one input**, which must be a **single string** formatted as JSON.
- The input string must contain exactly three fields: `"to"`, `"subject"`, and `"body"`.

Correct format:
send_email('{"to": "recipient@example.com", "subject": "Your Subject", "body": "Your message content."}')

Do NOT pass multiple arguments.
Do NOT omit the "to" field or rename it.

Always wrap the entire JSON in single quotes, so it is passed as one string.

Do not use positional arguments. Only use named arguments.
"""
class EmailInput(BaseModel):
    input_str: str = Field(description="Input Format")

@tool(args_schema=EmailInput)
def send_email(input_str: str) -> str:
    """Send an email to a recipient with subject and body."""
    try:
        data = json.loads(input_str)
        to = data["to"]
        subject = data["subject"]
        body = data["body"]
    except (json.JSONDecodeError, KeyError) as e:
        return f"Invalid input format or missing fields: {e}"

    sender_email = os.getenv("EMAIL_USER")
    password = os.getenv("EMAIL_PASSWORD")

    if not sender_email or not password:
        return "Email credentials are not set in environment variables."

    try:
        msg = MIMEMultipart()
        msg["From"] = sender_email
        msg["To"] = to
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain"))

        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, password)
            server.sendmail(sender_email, to, msg.as_string())

        return f"Email sent to {to} with subject '{subject}'."
    except Exception as e:
        return f"Failed to send email: {e}"


email_agent = initialize_agent(
    tools=[send_email],
    llm=llm_gemini,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    agent_kwargs={"system_message": system_message_email},
    verbose=True
)

# Agent as a Tool
email_agent_tool = Tool(
    name="email_agent_tool",
    func=email_agent.invoke,
    description=(
        "Send an email. Use when the user asks to contact or notify someone. "
        "Inputs must include: 'to' (email address), 'subject' (email title), and 'body' (message content)."
        )
)


C:\Users\PC\AppData\Local\Temp\ipykernel_12836\836779940.py:67: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  email_agent = initialize_agent(


In [6]:
result = email_agent_tool.run("Please send an email to test@example.com with subject 'Reminder' and body 'Don't forget the meeting at 10AM.'")
print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "send_email",
  "action_input": {
    "to": "test@example.com",
    "subject": "Reminder",
    "body": "Don't forget the meeting at 10AM."
  }
}
```
Observation: Email sent to test@example.com with subject 'Reminder'
Thought:Okay, I understand.

Action:
```json
{
  "action": "Final Answer",
  "action_input": "OK. I have sent the email."
}
```

> Finished chain.
{'input': "Please send an email to test@example.com with subject 'Reminder' and body 'Don't forget the meeting at 10AM.'", 'output': 'OK. I have sent the email.'}


In [7]:
email_agent.invoke('Send an email to kingdaica25@gmail.com with subject "Welcome" and body "Glad to have you on the team!"')



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "send_email",
  "action_input": {
    "to": "kingdaica25@gmail.com",
    "subject": "Welcome",
    "body": "Glad to have you on the team!"
  }
}
```
Observation: Email sent to kingdaica25@gmail.com with subject 'Welcome'
Thought:I have sent the email as requested.
Action:
```
{
  "action": "Final Answer",
  "action_input": "Email sent to kingdaica25@gmail.com with subject 'Welcome' and body 'Glad to have you on the team!'"
}
```

> Finished chain.


{'input': 'Send an email to kingdaica25@gmail.com with subject "Welcome" and body "Glad to have you on the team!"',
 'output': "Email sent to kingdaica25@gmail.com with subject 'Welcome' and body 'Glad to have you on the team!'"}

## This is the agent that use that agent as a tool

In [10]:
system_message = """
THINK AND ACTION, NOT ONLY THINKING
You are an intelligent assistant designed to solve tasks using available tools.
Do not answer directly after the first thought. Always reflect, plan, and use the tools as needed before answering.
TRUST the email returned by the graph tool as the correct email for that person, even if the email address does not contain their name
You have access to the following tools:
- get_graph_context – a graph database tool that helps you find people, their projects, and their email addresses.
Use the tool 'get_graph_context' when the user asks to find people information
Its input is a natural language asking about the user information

Use the tool 'send_email' when the user asks to contact someone.
Make sure to call the tool using the format:
send_email(to="...", subject="...", body="...")

Do not use positional arguments. Only use named arguments.

When calling a tool, always keep the action_input question as simple and specific as possible. Avoid long or complex sentences—use direct questions that clearly describe the goal.
You are an assistant that must use tool results exactly as returned.
NEVER assume values like email addresses or names. ALWAYS extract them from tool output.
If the tool returns an email, use it exactly. Do not guess or make up new ones.

Do not stop or finalize your answer until:
- You have used every necessary tool to fulfill the full request.
- You have answered all parts of the user's question.

When the user asks to contact someone:
1. First use the graph tool to get their email.
2. Then use the email tool to send the message.
3. Only proceed to the email step if the email address is found.
4. Do not repeat queries or enter infinite loops.
"""


# List of tools the agent can use
tools = [get_graph_context, send_email]

agent = initialize_agent(
    tools=tools,
    llm=llm_gemini,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    # , STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION , OPENAI_MULTI_FUNCTIONS , CHAT_ZERO_SHOT_REACT_DESCRIPTION, OPENAI_FUNCTIONS
    agent_kwargs={
        "system_message": SystemMessage(content=system_message)
    },
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    verbose=True
)

# Example query
response = agent.invoke('Find Alice email and send her an email with subject "Welcome" and body "Glad to have you on the team!"') #   
print(response)



> Entering new AgentExecutor chain...
Thought: I need to find Alice's email address first.
Action:
```
{
  "action": "get_graph_context",
  "action_input": {
    "question": "What is Alice's email address?"
  }
}
```

> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name: "Alice"})
RETURN p.email

Full Context:
[{'p.email': 'kingsanwano1@gmail.com'}]

> Finished chain.

Observation: Alice's email address is kingsanwano1@gmail.com.
Thought:I have Alice's email address. Now I can send her the email.
Action:
```
{
  "action": "send_email",
  "action_input": {
    "to": "kingsanwano1@gmail.com",
    "subject": "Welcome",
    "body": "Glad to have you on the team!"
  }
}
```
Observation: Email sent to kingsanwano1@gmail.com with subject 'Welcome'
Thought:Question: I have sent the email to Alice.
Thought: I need to respond with a JSON blob that specifies the "Final Answer".
Action:
```json
{
  "action": "Final Answer",
  "action_input": "I have sent the 